# Dynamic GCP calibration

Notebook which computes and saves results of the Dynamic GCP calibration

In [ ]:
import numpy as np
import importlib
import matplotlib.pyplot as plt
import scripts.theodolite_utils as theodo_u
import scripts.theodolite_function as theodo_f
import scripts.resection_functions as theodo_r
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# This cell gathers result for the dynamic control points for the graphic below
# With a linear interpolation

theodo_r = importlib.reload(theodo_r)
theodo_u = importlib.reload(theodo_u)
theodo_f = importlib.reload(theodo_f)

path_list = [
    "../data/20220711-1/",
    "../data/20220715-2/",
    "../data/20220715-3/",
    "../data/20220717-5/",
    "../data/20220910-1/",
    "../data/20220910-2/",
    "../data/20220910-3/",
    "../data/20220910-4/",
    "../data/20220910-5/",
    "../data/20220910-6/",
]

path_list_inter = [
    ["../data/20220711-2/"],
    [""],
    [""],
    ["../data/20220717-2/","../data/20220717-3/","../data/20220717-4/"],
    ["../data/20220910-7/","../data/20220910-8/","../data/20220910-9/"],
    ["../data/20220910-7/","../data/20220910-8/","../data/20220910-9/"],
    ["../data/20220910-7/","../data/20220910-8/","../data/20220910-9/"],
    ["../data/20220910-7/","../data/20220910-8/","../data/20220910-9/"],
    ["../data/20220910-7/","../data/20220910-8/","../data/20220910-9/"],
    ["../data/20220910-7/","../data/20220910-8/","../data/20220910-9/"],
]

save_results = False

velocity_outlier = [
    2.2,2.2,2.2,2.2,1.2,
    1.2,1.2,1.2,1.2,1.2
]

errors_linear_dynamic = []
errors_linear_dynamic_cp = []
errors_linear_dynamic_cp_exp = []
TF_list_linear_dynamic = []
for i,j,k in zip(path_list,path_list_inter,velocity_outlier):
    # Generate file path
    file_prediction_dynamic = theodo_u.if_file_exist(i + 'filtered_prediction/f-2-1-1-1-6-0-L_','1.csv')
    file_GCP = theodo_u.if_file_exist(i + 'total_stations/GCP.txt','')
    print("don't give up: ", i)
    Tf_loop = []
    error_dynamic_cp_loop = []
    error_dynamic_exp_loop = []
    error_dynamic_cp_exp_loop = []
    for m in j:
        file_prediction_inter = theodo_u.if_file_exist(m + 'filtered_prediction/f-2-1-1-1-6-0-L_','1.csv')
        file_sensors = theodo_u.if_file_exist(m + "sensors_extrinsic_calibration/calibration_results.csv",'')
        extrinsic_calibration_results = theodo_u.read_extrinsic_calibration_results_file(file_sensors)
        errors_dynamic, errors_dynamic_cp, errors_dynamic_cp_exp, TF_list = theodo_r.dynamic_control_points_error_comparison(dynamic_file_path=file_prediction_dynamic, exp_file_path=file_prediction_inter, marker_file_path=file_GCP, inter_prism_dist=extrinsic_calibration_results,
                                            rate = 10, velocity_outlier = k)
        print("Cp error: ",np.median(errors_dynamic_cp),np.std(errors_dynamic_cp))
        print("Inter-prism: ", np.median(errors_dynamic_cp_exp),np.std(errors_dynamic_cp_exp))
        error_dynamic_exp_loop += errors_dynamic
        error_dynamic_cp_loop += errors_dynamic_cp
        error_dynamic_cp_exp_loop += errors_dynamic_cp_exp
        Tf_loop += TF_list
    errors_linear_dynamic += error_dynamic_exp_loop
    errors_linear_dynamic_cp += error_dynamic_cp_loop
    errors_linear_dynamic_cp_exp += error_dynamic_cp_exp_loop
    TF_list_linear_dynamic += TF_list
    if(save_results):
        theodo_u.save_error_list_to_file(error_dynamic_cp_loop, i+"list_error/errors_linear_dynamic_cp_cp.csv")
        theodo_u.save_error_list_to_file(error_dynamic_cp_exp_loop, i+"list_error/errors_linear_dynamic_cp_experiment.csv")
        theodo_u.save_tf_list_to_file_multi(TF_list, i+"list_tf/TF_list_linear_dynamic_cp.csv")
print("End !")

In [ ]:
# Display results according to different metrics
%matplotlib notebook

fig = plt.figure(figsize =(10, 5))
ax = fig.add_subplot(111)
lis = [errors_linear_dynamic_cp,errors_linear_dynamic_cp_exp]

box = ax.boxplot(lis, notch=True, patch_artist=True, vert = 1, showfliers=False, showmeans=False)
plt.xticks([1,2],
               ["GCP\n\n\n M: "+str(round(np.median(lis[0]),2))+"mm \n Std: "+str(round(np.std(lis[0]),2))+"mm\n"+str(round(len(lis[0])/3))+" pts",
            "Inter-prism\n\n\n M: "+str(round(np.median(lis[1]),2))+"mm \n Std: "+str(round(np.std(lis[1]),2))+"mm\n"+str(round(len(lis[1])/3))+" pts",
           ])
colors_box = ['#069AF3','#069AF3']

for patch, color in zip(box['boxes'], colors_box):
    patch.set_facecolor(color)

ax.set_ylabel("Error [mm]")
fig.tight_layout()
plt.show()